<a href="https://colab.research.google.com/github/anjunjin/PFE-ING3-IA/blob/master/LinkWithTwitter.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Installation de la library *python-dotenv*

In [1]:
!pip install python-dotenv

Importation des libraries

In [2]:
import tweepy
import pickle
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

import pandas as pd
from tqdm import tqdm
import numpy as np
import matplotlib.pyplot as plt
from dotenv import dotenv_values
import requests
from IPython.display import JSON

Liaison au Google Drive

In [3]:
from google.colab import drive
drive.mount('/content/drive')
DATA_PATH = "/content/drive/My Drive/PFE/"

Mounted at /content/drive


Récupération dans ***data*** des donnés nécessaires de tableauSites.csv

In [4]:
data = pd.read_csv(DATA_PATH+"tableauSites.csv", encoding='utf-8').drop(['Unnamed: 0', 'id'], axis=1).values.tolist()

Récupération des clés pour se connecter au compte Twitter Developers et utiliser son API

In [5]:
API_TOKEN = pd.read_csv(DATA_PATH+"Clés.csv", encoding='utf-8', sep=',')['Bearer Token'][0]
API_KEY = pd.read_csv(DATA_PATH+"Clés.csv", encoding='utf-8', sep=',')['API Key'][0]
API_SECRET = pd.read_csv(DATA_PATH+"Clés.csv", encoding='utf-8', sep=',')['API Key Secret'][0]

Authentifiation au compte Twitter Developers et récupération de l'API tweepy

In [6]:
auth = tweepy.AppAuthHandler(API_KEY, API_SECRET)
api = tweepy.API(auth, wait_on_rate_limit=True)

Création d'une classe TwitterUser contenant les fonctions de récupération de données (utilisateurs, tweets, ...)

In [7]:
    class TwitterUser():
        _screen_name = ""
        _id = ""
        _user_data = None
        followers = []
        following = []
        tweet = []
        like = []
        request_count = 0

        def __init__(self, sname = "", _json = {}):
            if (sname != ""):
                self._screen_name = sname
                self.get_info()
            if (_json != {}):
                self._json = _json

        def get_info(self):
            self._user_data = api.get_user(screen_name=self._screen_name)

        def get_all_data(self):
            self.get_tweets()
            # self.get_followers()
            # self.get_following()
            # self.get_like()

        def set_name(self, name):
            self._screen_name = name

        def set_id(self, _id):
            self._id = _id

        def get_followers(self):
            with tqdm(total=self._user_data.followers_count, desc="get_followers for " + self._user_data.screen_name) as pbar:
                for page in tweepy.Cursor(api.followers, screen_name=self._screen_name, count=200).pages():
                    self.followers.extend(page)
                    pbar.update(len(page))


        def get_following(self):
            with tqdm(total=self._user_data.friends_count, desc="get_following for " + self._user_data.screen_name) as pbar:
                for page in tweepy.Cursor(api.friends, screen_name=self._screen_name, count=200).pages():
                    self.following.extend(page)
                    pbar.update(len(page))

        # on ne peut prendre que les 3000 derniers
        def get_tweets(self):
            if self._user_data.protected:
                print("can't get tweets from " + self._user_data.screen_name + ", the account is protected")
            else:
                with tqdm(total=3000, desc="get_tweets for " + self._user_data.screen_name) as pbar:
                    for page in tweepy.Cursor(api.user_timeline, screen_name=self._screen_name, count=200).pages():
                        self.tweet.extend(page)
                        pbar.update(len(page))

        # on ne peut prendre que les 3000 derniers
        def get_like(self):
            if self._user_data.protected:
                print("can get like from " + self._user_data.screen_name + ", the account is protected")
            else:
                with tqdm(total=3000, desc="get_likes for " + self._user_data.screen_name) as pbar:
                    for page in tweepy.Cursor(api.favorites, screen_name=self._screen_name, count=200).pages():
                        self.like.extend(page)
                        pbar.update(len(page))


Sauvegarde des données reçu pour chaque utilisateur Twitter recherché

In [8]:
import pickle

with open(DATA_PATH+"PaucBas.pickle", "ab") as obj_file:
  for site in data:
    user = TwitterUser(sname= site[2])
    user.get_all_data()
    pickle.dump([user], obj_file, pickle.HIGHEST_PROTOCOL)
    # print(fake_news_twitter[site])

get_tweets for france_soir: 3213it [00:06, 507.66it/s]
get_tweets for ThierryRegenere:  12%|█▎        | 375/3000 [00:00<00:05, 507.61it/s]


TweepError: ignored